In [1]:
import pandas as pd
import plotly.express as px

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [2]:
def check_missing_columns(df: pd.DataFrame) -> pd.Series:
    """Check if any of the columns in the dataframe contains missing values.

    Args:
        df (pd.DataFrame): dataframe to check.

    Returns:
        pd.Series: series of boolean values to denote whether the column contains missing values.
    """
    return df.isnull().any()

## Perform Sanity Checks

### Check for missing values in the datasets

Check if the train dataset contains any missing values in its columns.

In [3]:
check_missing_columns(train)

id                False
industry           True
state             False
request_date      False
term              False
employee_count    False
business_new      False
business_type     False
location          False
other_loans       False
loan_amount       False
insured_amount    False
default_status    False
dtype: bool

Industry column in train dataset contains missing values. Let's dive deeper to see how many records contains missing industry.

In [4]:
train[train.industry.isna()]

,id,industry,state,request_date,term,employee_count,business_new,business_type,location,other_loans,loan_amount,insured_amount,default_status
1946,3771775001,NaN,NH,20-Nov-09,12,1,New,0,Rural,N,$100.00,"$75,000.00",0


Let's replace that missing value with 'Unknown'

In [5]:
train.industry = train.industry.fillna('Unknown')


Check if the test dataset contains any missing values in its columns.

In [6]:
check_missing_columns(test)

id                False
industry          False
state             False
request_date      False
term              False
employee_count    False
business_new      False
business_type     False
location          False
other_loans       False
loan_amount       False
insured_amount    False
dtype: bool

Test dataset does not contain any missing values.

### Check if the test dataset contains categorical values that are not present in train dataset.

In [7]:
def check_categories(test: pd.DataFrame, train: pd.DataFrame, categorical_fields: list = []) -> list:
    """Check if test dataset contains categorical values not present in train dataset. 

    Args:
        test (pd.DataFrame): test dataset.
        train (pd.DataFrame): train dataset.
        categorical_fields (list, optional): columns to check. Defaults to [].

    Returns:
        list: list of columns in test dataset which contains categorical values not present in train dataset
    """
    columns_missing_categories = []

    for field in categorical_fields:
        unique_train_categories = set(train[field].unique())
        unique_test_categories = set(test[field].unique())

        print(f'Checking categorical column: {field}')
        print(f'Unique categories in train: {unique_train_categories}')
        print(f'Unique categories in test: {unique_test_categories}')

        missing_values = unique_test_categories - unique_train_categories
        print(f'Categories in test but not train: {missing_values}\n')

        if missing_values:
            columns_missing_categories.append(field)
    
    return columns_missing_categories

In [8]:
columns_missing_categories = check_categories(
    test, train, 
    categorical_fields = ['industry', 'state', 'business_new', 'business_type', 'location', 'other_loans']
)
columns_missing_categories

Checking categorical column: industry
Unique categories in train: {'Others', 'Energy', 'Transportation', 'Healthcare', 'Engineering', 'Hotel', 'Trading', 'Agriculture', 'Education', 'Construction', 'Entertainment', 'Finance', 'Real Estate', 'Manufacturing', 'Consulting', 'Administration', 'Unknown'}
Unique categories in test: {'Others', 'Energy', 'Transportation', 'Healthcare', 'Hotel', 'Engineering', 'Trading', 'Agriculture', 'Construction', 'Finance', 'Entertainment', 'Education', 'Real Estate', 'Consulting', 'Manufacturing', 'Administration'}
Categories in test but not train: set()

Checking categorical column: state
Unique categories in train: {'RI', 'NJ', 'NE', 'SD', 'OK', 'MI', 'SC', 'CO', 'TN', 'KY', 'IA', 'FL', 'WA', 'DE', 'NY', 'WI', 'WV', 'MN', 'AZ', 'MT', 'AK', 'LA', 'KS', 'NM', 'IN', 'ME', 'ID', 'MA', 'UT', 'ND', 'PA', 'CA', 'TX', 'NH', 'MS', 'MD', 'OR', 'HI', 'NV', 'CT', 'AR', 'MO', 'VT', 'OH', 'AL', 'WY', 'VA', 'GA', 'NC', 'IL'}
Unique categories in test: {'NJ', 'NE', 'SD

[]

All of the categorical values present in test dataset can also be found in the train dataset. Additionally, it could be observed that the location field only has one value 'Rural' in both the train and test datasets. Since every record has the same location 'Rural', the location field would not have any discriminative power over the target variable default_status. Hence, let's drop the location field from the potential list of features to include in the model.

In [89]:
train = train.drop(columns = ['location'])
test = test.drop(columns = ['location'])

## Perform Data Preprocessing

### Preprocessing Steps:

1) Cast request_date to a datetime object.
2) Cast loan_amount and insured_amount to float.

In [9]:
def convert_to_datetime(df: pd.DataFrame, datetime_col: str = 'request_date', format: str = '%d-%b-%y') -> pd.DataFrame:
    """Convert date string to datetime object in column.

    Args:
        df (pd.DataFrame): dataframe containing at least one date string column.
        datetime_col (str, optional): name of date string column. Defaults to 'request_date'.
        format (str, optional): format of date string. Defaults to '%d-%b-%y'.

    Returns:
        pd.DataFrame: processed dataframe with date string casted to datetime object.
    """
    df[datetime_col] = pd.to_datetime(df[datetime_col], format = format)
    return df

In [10]:
train = convert_to_datetime(train)
test = convert_to_datetime(test)

In [13]:
def convert_amt_cols_to_float(df: pd.DataFrame, amt_cols: list = ['loan_amount', 'insured_amount']) -> pd.DataFrame:
    """Convert currency string columns to float

    Args:
        df (pd.DataFrame): dataframe containing currency string columns.
        amt_cols (list, optional): currency string columns. Defaults to ['loan_amount', 'insured_amount'].

    Returns:
        pd.DataFrame: processed dataframe with currency string columns casted to float.
    """
    df = df.copy()
    
    for col in amt_cols:

        # Check that each value in the amount column is a string
        types = df[col].apply(type).value_counts()
        print(types)
        assert len(types) == 1
        assert str(types.index[0]) == "<class 'str'>"

        # Remove $ and , characters
        df[col] = df[col].str.replace(r'[^0-9\.]', '', regex = True).astype(float)

        # Perform sanity check to check if there are any np.nan values after conversion
        assert not df[col].isna().any()

        # Compute the statistics of the values
        print(f'{df[col].describe()}\n')

    return df

In [14]:
train = convert_amt_cols_to_float(train, amt_cols = ['loan_amount', 'insured_amount'])
test = convert_amt_cols_to_float(test, amt_cols = ['loan_amount', 'insured_amount'])

loan_amount
<class 'str'>    2402
Name: count, dtype: int64
count    2.402000e+03
mean     2.044877e+05
std      3.643356e+05
min      1.000000e+02
25%      2.500000e+04
50%      5.000000e+04
75%      2.164925e+05
max      4.000000e+06
Name: loan_amount, dtype: float64

insured_amount
<class 'str'>    2402
Name: count, dtype: int64
count    2.402000e+03
mean     1.550167e+05
std      3.114227e+05
min      1.700000e+03
25%      1.275000e+04
50%      3.500000e+04
75%      1.250000e+05
max      4.000000e+06
Name: insured_amount, dtype: float64

loan_amount
<class 'str'>    601
Name: count, dtype: int64
count    6.010000e+02
mean     1.885422e+05
std      3.085025e+05
min      2.000000e+03
25%      2.500000e+04
50%      5.194000e+04
75%      2.180000e+05
max      2.000000e+06
Name: loan_amount, dtype: float64

insured_amount
<class 'str'>    601
Name: count, dtype: int64
count    6.010000e+02
mean     1.469891e+05
std      2.720625e+05
min      1.000000e+03
25%      1.275000e+04
50%      3

## Perform Feature Engineering 

### Features to create

1. loan_insured_amount_diff: Difference between loan_amount and insured_amount (i.e. loan_amount - insured_amount)
2. insured_loan_ratio: Ratio between insured_amount and loan_amount (i.e. insured_amount / loan_amount)

In [15]:
def create_loan_insured_features(df: pd.DataFrame) -> pd.DataFrame:
    """Create loan_insured_amount_diff and insured_loan_ratio features.

    Args:
        df (pd.DataFrame): dataframe containing loan_amount and insured_amount columns.

    Returns:
        pd.DataFrame: processed dataframe with loan_insured_amount_diff and insured_loan_ratio features added.
    """
    df['loan_insured_amount_diff'] = df['loan_amount'] - df['insured_amount']
    df['insured_loan_ratio'] = df['insured_amount'] / df['loan_amount']
    return df

In [16]:
train = create_loan_insured_features(train)
test = create_loan_insured_features(test)

## Perform Exploratory Data Analysis

### Check the request date distributions in train and test datasets.

In [17]:
train.request_date.describe()

count                             2402
mean     2010-03-14 23:25:49.708576256
min                2009-10-01 00:00:00
25%                2009-12-14 00:00:00
50%                2010-03-11 00:00:00
75%                2010-06-01 18:00:00
max                2010-09-30 00:00:00
Name: request_date, dtype: object

In [18]:
test.request_date.describe()

count                              601
mean     2010-03-11 00:38:20.166389248
min                2009-10-01 00:00:00
25%                2009-12-09 00:00:00
50%                2010-03-04 00:00:00
75%                2010-06-07 00:00:00
max                2010-09-30 00:00:00
Name: request_date, dtype: object

The request_date range for both the train and test datasets are the same, starting from 2009-10-01 and ending on 2010-09-30. request_date wise, it seems that both the train and test datasets were drawn from the same distribution.

### Check the distribution of categorical features and target variable

In [83]:
def plot_categories_distribution(df: pd.DataFrame, category_col: str, title: str = None, width: int = 500, height: int = 800):
    """Plot distribution of categorical values.

    Args:
        df (pd.DataFrame): dataframe containing categorical columns.
        category_col (str): name of categorical column.
        title (str): title of plot.
        width (int, optional): width of plot. Defaults to 500.
        height (int, optional): height of plot. Defaults to 800.

    Returns:
        Figure: distribution plot of categorical values.
    """
    title = f'Percentage of each {category_col} in train dataset' if title is None else title
    fig = px.bar(
        (df[category_col].astype(str).value_counts() / df.shape[0] * 100).sort_values().reset_index().rename(columns = {'count': 'percentage'}), 
        x = 'percentage', y = category_col, title = title
    )
    fig.update_yaxes(tickmode = 'linear')
    fig.update_layout(width = width, height = height)
    return fig

In [84]:
plot_categories_distribution(train, 'industry')

In [85]:
plot_categories_distribution(train, 'state')

In [86]:
plot_categories_distribution(train, 'business_new')

In [87]:
plot_categories_distribution(train, 'business_type')

In [90]:
plot_categories_distribution(train, 'other_loans')

In [91]:
plot_categories_distribution(train, 'default_status')

It could be observed that the train dataset is imbalanced in terms of the target variable, default status. The number of customers that do not default (i.e. default_status = 0) is ~2x of the number of customers that default (i.e. default_status = 1).

In [ ]:
# TODO:
# 1) Sanity check for missing values in train, test (Done)
# 2) Sanity check for categorical values that exist in test but not in train (Done)
# 3) Cast 'request_date' to pandas datetime (Done)
# 4) Cast 'loan_amount' and 'insured_amount' to float (Done)
# 5) Create new feature, 'loan_insured_amount_diff', 'loan_amount' - 'insured_amount' + ratio (2 features) (Done)
# 6) Check request_date distribution, i.e. min and max date (I don't think we should generate features based on this, but if really want, we can generate day of week (1 - 7), day of month (1 - 31), week number (1 - 52), month (1 - 12), year (2009 - 2010)) (KIV for now) (Done)
# 7) Check probability distribution of categorical features ('industry', 'state', 'business_new', 'business_type', 'other_loans', 'default_status') (6 bar plots?) (Done)

# 8) Check distribution of 'term', 'employee_count', 'loan_amount', 'insured_amount', 'loan_insured_amount_diff', 'insured_loan_ratio' (df.describe) (5 + 5 box/violin plots?)

# 9) Plot probability of default_status vs each categorical value (in 1 plot) for each categorical feature (5 bar plots)
# 10) Plot distribution of numerical feature for each default_status (5 + 5 bar/violin plots?)
# 11) Encode categorical features with LabelEncoder ('industry', 'state', 'business_new', 'business_type', 'location', 'other_loans')
# 12) Add steps 3, 4, 5, 6, 11 to pipeline for data preprocessing before ingestion into model for inference

# 13) Perform stratified k-fold validation to estimate the final number of training iterations and the average model performance on full dataset; total 16 features
# 14) Train model on full dataset based on the final number of training iterations derived from step 13
# 15) Use SHAP values to explain the contribution of each feature on the predictions (train/test dataset) + generate feature importance
# 16) Perform probability calibration (i.e. Platt Scaling/Isotonic Regression via CalibratedClassifierCV class) on the trained LightGBM model as the scores returned by the model is not true probability; have to perform this step in each fold of k-fold cross validation
# 17) Calculate ROC AUC based on calibrated probabilities
# 18) Plot ROC AUC/Precision Recall curve to determine the optimal probability threshold to set to predict default_status 1
# 19) Try multiple models (i.e Logistic Regression, LinearSVC) as part of model selection?
# 20) Feature selection? Feature importance based on SHAP values
# 21) A presentation to share your findings and takeaways to your non-technical business stakeholders to convince them that your approach is correct, effective, and deployable. 